# ModOAP - Téléchargement des Illustrations des documents Gallica

Ce carnet propose de télécharger les illustrations de documents Gallica.

Les illustrations sont repérées par le processus d'OCR utilisé par Gallica. Les documents doivent donc être disponibles "en mode texte" sur Gallica.

Le carnet télécharge les illustrations du document au format JPG, et référence les informations des illustrations dans un fichier structuré au format JSON.

Les identifiants ark des docs à télécharger peuvent être listés dans une colonne d'un fichier .xlsx présent sur le drive. 
Le dossier exemple_liste_identifiants contient un exemple de fichier .xlsx listant 5 documents Gallica.

In [ ]:
 #@markdown ### Préparation et connexion à un compte Google Drive
#@markdown Lancer cette cellule, puis cliquer sur le lien généré par Google pour connecter un compte Drive si demandé.

import os
from google.colab import drive
# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

import requests
from openpyxl import load_workbook
import urllib.request, urllib.error, urllib.parse
from urllib.error import HTTPError, URLError

import json
from bs4 import BeautifulSoup
try :
  import xmltodict
except :
  !pip -q install xmltodict
  import xmltodict
import unicodedata
import re 
import glob
from tqdm import tqdm

def remove_accents(s):
  # In : chaine avec caractères diachrités
  # Out : chaine sans accent
  return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def normalisation_titre(titre):
  # In : chaine titre
  # Out : chaine titre pour nom de fichier
  titre = remove_accents(titre)
  titre = re.sub('[^a-zA-Z0-9- ]', '', titre)
  titre = re.sub('[ ]', '_', titre)
  return "_".join(titre.split("_")[:6])

def get_var_tb(tb) :
  # In : textblock (ordered dict key, dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  if ratio_width != 1.0 :
    width = int(tb["@WIDTH"] * ratio_width)
    hpos = int(tb["@HPOS"] * ratio_width)
  else :
    width = tb["@WIDTH"]
    hpos = tb["@HPOS"]

  if ratio_height != 1.0 :
    height = int(tb["@HEIGHT"] * ratio_height) 
    vpos = int(tb["@HEIGHT"] * ratio_height)
  else :
    height = tb["@HEIGHT"]
    vpos = tb["@VPOS"]  
  id = tb["@ID"]
  return width, height, hpos, vpos, id

def get_tb(tb_entry) :
  # In : textblock (ordered dict key, list ou dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  if isinstance(tb_entry, list) :
    for tb in tb_entry :
      width, height, hpos, vpos, id = get_var_tb(tb)
  else : 
    width, height, hpos, vpos, id = get_var_tb(tb_entry)
  return width, height, hpos, vpos, id

def block_treatment(tb, num_page,titre,date_pub,dossier_sortie,taille_images) :
  # In :  textblock (ordered dict key), numéro de page (int)
  # lance get_tb
  # ajoute la position et le contenu du bloc texte dans un dictionnaire
  width, height, hpos, vpos, id = get_tb(tb)
  position_dic = {"width" : width, "height" : height, "hpos" : hpos, "vpos" : vpos}
  dico_id = {"Page_Num" : num_page, "Position" : position_dic}
  text_blocks[id] = dico_id
  url = "https://gallica.bnf.fr/iiif/ark:/12148/{}/f{}/{},{},{},{}/{}/0/native.jpg".format(ark,num_page,hpos,vpos,width,height,taille_images)
  nomfichier = date_pub+"_"+titre+"_"+id+".jpg"
  cheminout = os.path.join(dossier_sortie,date_pub+"_"+titre,nomfichier)

  try :
    urllib.request.urlretrieve(url, cheminout)
  except (HTTPError, URLError) as erreur:
    print(str(erreur.reason))     

def VerificationTailleImage(ark,num_page="1"):
  s = requests.get("https://gallica.bnf.fr/iiif/ark:/12148/{}/f{}/info.json".format(ark,num_page), stream=True)
  dicinfos = json.loads(s.text)
  width = dicinfos["width"]
  height = dicinfos["height"]
  return width, height


def infos_doc(ark):
  # In :  identifiant ark
  # Out : titre, date de publication du document (str)
  url_biblio = "https://gallica.bnf.fr/services/OAIRecord?ark="+ark
  s = requests.get(url_biblio, stream=True)
  bibliodico = xmltodict.parse(s.text)
  try :
    titre = bibliodico["results"]["title"]
  except :
    titre = "unknown"
  try :
    date_pub = bibliodico["results"]["date"]["#text"]
  except :
    date_pub = "unknown"
  return titre, date_pub
  
def nombre_pages(ark):
  # In :  identifiant ark
  # Out : nombre de pages (int)
  PAGINATION_BASEURL = 'https://gallica.bnf.fr/services/Pagination?ark='
  url = "".join([PAGINATION_BASEURL, ark])
  s = requests.get(url, stream=True)
  paginationdic = xmltodict.parse(s.text)
  nb_pages = int(paginationdic["livre"]["structure"]["nbVueImages"])
  return nb_pages

def ark_processing(ark, dossier_sortie,taille_images) :
  # In :  identifiant ark
  # Out : fichier json contenant : 
    # titre, date, nombre de pages du document
    # id, position, contenu des blocs de textes de chaque page
  flag = 0
  titre, date_pub = infos_doc(ark)
  titre_fichier = normalisation_titre(titre)
  if not os.path.exists(os.path.join(dossier_sortie,date_pub+"_"+titre_fichier)):
    os.makedirs(os.path.join(dossier_sortie,date_pub+"_"+titre_fichier))
  try :
    nombre_pagess = nombre_pages(ark)
    print(nombre_pagess)
  except :
    print("Pagination indisponible, document non-téléchargé : ", ark)
    

  print()
  print("Titre du document : ", titre)
  print("Dossier de destination : ", dossier_sortie)
  print("Téléchargement des illustrations du document :")
  info_doc = {"Titre" : titre, "Publication_Date" : date_pub, "Total_Pages" : nombre_pagess }
  global text_blocks 
  text_blocks = {}
  # Pour chaque page du document :
  for num_page in tqdm(range(1,nombre_pagess+1)) :
    # Transforme le fichier OCR ALTO de la page en un dictionnaire :
    alto_url = 'https://gallica.bnf.fr/RequestDigitalElement?O='+ark+'&E=ALTO&Deb='+str(num_page)
    s = requests.get(alto_url, stream=True)
    altodic = xmltodict.parse(s.text)
    if flag == 0 :
      global ratio_width
      global ratio_height 
      width_serveur,height_serveur = VerificationTailleImage(ark)
      ratio_width = int(width_serveur) / int(altodic["alto"]["Layout"]["Page"]["@WIDTH"]) 
      ratio_height = int(height_serveur) / int(altodic["alto"]["Layout"]["Page"]["@HEIGHT"])
      
      altodic["alto"]["Layout"]["Page"]["@HEIGHT"]
      
      info_doc["Page_hauteur"] = altodic["alto"]["Layout"]["Page"]["@HEIGHT"]
      info_doc["Page_largeur"] = altodic["alto"]["Layout"]["Page"]["@WIDTH"]
      flag = 1
    else : pass
    # Si un ou plusieurs blocs de texte sont directement présents dans le PrintSpace : 
    try :
      tb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["Illustration"]
      if isinstance(tb_entry, list) :
        for tb in tb_entry :
          block_treatment(tb, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
        
      else :
        block_treatment(tb_entry, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
    except :
      pass
    # Si un ou plusieurs blocs composés sont présents dans le PrintSpace : 
    try :
      cb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["ComposedBlock"]
      if isinstance(cb_entry, list) :
        for cb in cb_entry :
          tb_entry = cb["Illustration"]
          if isinstance(tb_entry, list) :
            for tb in tb_entry :
              block_treatment(tb, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
          else :
            block_treatment(tb_entry, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
      else :
        tb_entry = cb_entry["Illustration"]
        if isinstance(tb_entry, list) :
          for tb in tb_entry :
            block_treatment(tb, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
        else :
          block_treatment(tb_entry, num_page,titre_fichier,date_pub, dossier_sortie,taille_images)
    except :
      pass
    # Création du fichier final :

  jsondic = {"Infos_Doc" : info_doc, "Illustrations" : text_blocks} 
  with open(os.path.join(dossier_sortie,date_pub+"_"+titre_fichier,ark+"_illustrations.json"), 'w') as jout :
    json.dump(jsondic, jout)

def bnf2gall(arkbnf):
  # In :  identifiant ark au format type cb453908509
  # Out : identifiant ark au format type bpt6k9799524x (consultable sur Gallica)
  url = "https://catalogue.bnf.fr/ark:/12148/"+str(arkbnf)
  s = requests.get(url, stream=True)
  html = BeautifulSoup(s.content) # html5lib
  for link in html.findAll('a', {'class': 'exemplaire-action-visualiser'}):
    ark = link['href'].split("/")[-1]
  return ark

In [ ]:
#@markdown ### A partir d'un identifiant ARK :
#@markdown Renseigner les paramètres avant de lancer la cellule

#@markdown ---
#@markdown Entrer l'identifiant ARK du document :

ark = "" #@param {type:"string"}

#@markdown Exemple d'identifiant:
#@markdown bpt6k9799524x ou cb453908509

#@markdown ---
#@markdown Entrer le chemin de destination où télécharger les illustrations du document :

dossier_sortie = ""#@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/Corpus/

if not os.path.exists(dossier_sortie):
  os.makedirs(dossier_sortie)

#@markdown ---

#@markdown #### Réduction de la taille des illustrations téléchargées :
#@markdown Déplacer le curseur pour sélectionner un pourcentage de la taille initiale des images à télécharger

#@markdown Le ratio hauteur/largeur est conservé

taille_images = 100 #@param {type:"slider", min:0, max:100, step:10}
if taille_images == 100 :
  taille_images = "full"
else :
  taille_images = "pct:"+str(taille_images)
  size = taille_images

if ark.startswith("cb") :
  ark = bnf2gall(ark)
ark_processing(ark, dossier_sortie,taille_images)


In [ ]:
#@markdown ### A partir d'un fichier tableur :

#@markdown Renseigner les paramètres avant de lancer la cellule

#@markdown ---
#@markdown Entrer le chemin vers un fichier tableur (xlsx ou xlsm) dont une colonne contient les identifiants ARK des documents :
chemin_fichier_xls = "" #@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/Document/documents.xlsx

#@markdown Possibilité de copier/coller le chemin depuis la fenêtre de gauche : onglet "Fichiers" -> clic droit sur un dossier -> "Copier le chemin"

#@markdown ---
#@markdown Préciser l'indice de la colonne du tableau contenant les identifiants ARK :
colonne_ark = "A" #@param {type:"string"}
#@markdown Exemple d'indice : A

#@markdown ---
#@markdown Entrer le chemin de destination où télécharger les illustrations du document :



dossier_sortie = ""#@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/Corpus/

#@markdown ---
#@markdown Réduction de la taille des illustrations téléchargées :

#@markdown Déplacer le curseur pour sélectionner un pourcentage de la taille initiale des images à télécharger :

#@markdown Le ratio hauteur/largeur est conservé


if not os.path.exists(dossier_sortie):
  os.makedirs(dossier_sortie)

taille_images = 100 #@param {type:"slider", min:0, max:100, step:10}
if taille_images == 100 :
  taille_images = "full"
else :
  taille_images = "pct:"+str(taille_images)
  size = taille_images

arks_doc = []

# Chargement du fichier xls
try :
  classeur= load_workbook(chemin_fichier_xls)
except :
  print("Le fichier xls n'a pas été chargé correctement")

for onglet in classeur.sheetnames:
  onglet_courant = classeur[onglet]
  colonne = onglet_courant[colonne_ark]
  for cellule in colonne :
    if str(cellule.value).startswith("http") or str(cellule.value).startswith("ark"):
      arks_doc.append(str(cellule.value).split("/")[-1].strip())
    elif str(cellule.value).startswith("cb") or str(cellule.value).startswith("bp"):
      arks_doc.append(str(cellule.value))

print("{0} liens récupérés dans {1} onglets".format(len(arks_doc), len(classeur.sheetnames)))

arks_doc = set(arks_doc)

for ark in arks_doc :
  if ark.startswith("cb") :
    try :
      ark = bnf2gall(ark)
      ark_processing(ark,dossier_sortie,taille_images)
    except :
      pass
  else :
    try :
      ark_processing(ark,dossier_sortie,taille_images)
    except :
      print("Le document {} n'a pas été téléchargé".format(ark))
      pass
